## Load Libraries

In [1]:
from __future__ import division
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
from xgboost import plot_importance
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix, mean_squared_error
import seaborn as sns
from statannot import add_stat_annotation
from sklearn.linear_model import Lasso, MultiTaskLasso, Ridge, ElasticNet

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression, f_regression

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.decomposition import PCA

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

from sklearn import metrics


from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import statsmodels.api as sm

import time

import scipy

from sklearn.svm import SVR

from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor

from statsmodels.stats.multitest import fdrcorrection

from statsmodels.stats import multitest

## Define functions

In [2]:
def get_color(var, X_cat_, color_dict_):

    X_factors = list(X_cat_.factors)
    X_name = list(X_cat_.newname)

    index_code = X_name.index(var)
    cat = X_factors[index_code]

    return color_dict_[cat]


def get_bar_colors(data_, X_cat_):
    color_dict = {}
    color_dict['FP'] = '#FAF85F'
    color_dict['HF'] = '#cc8963' #b55d60
    color_dict['FF1'] = '#FF9E0D'
    color_dict['CF'] = '#b55d60' #cc8963
    color_dict['FF'] = '#5975a4'
    color_dict['DF'] = '#5f9e6e'
    color_dict['FlairFL'] = '#DAEB07'
    color_dict['FlairFR'] = '#EB9700'
    color_dict['FlairF'] = '#FFD400'
    color_dict['GMF'] = '#757575'
    color_dict['WMF'] = '#DADEE0'

    X_factors = list(X_cat_.factors)
    X_name = list(X_cat_.name)

    bar_color_total = []
    for i in list(data_.Features):
        if(i == 'Country'):
            bar_color_total.append(color_dict['Country'])
            continue
        index_code = X_name.index(i)
        cat = X_factors[index_code]
        bar_color_total.append(color_dict[cat])
    return bar_color_total



def plot_estimate_value_v1(regression_model, X_cat_ = [], title = '',  xlim =[0, 2] ,fig_size = (8,12), size = 12, pvalue_type = 'False', plot_ci = '', report_pos = [50, 4], n = 10):

    df = regression_model
    df.index.name = 'Features'
    df = df.iloc[1:-5, 0:-1]

    for i in range(3):
            #print(i, X_RAW_edu_level[X_RAW_edu_level.columns[i]].dtype)
        df[df.columns[i]] = np.abs(pd.to_numeric(df[df.columns[i]],errors = 'coerce'))

    df = df.reset_index()
    data = df.sort_values('Estimate', ascending=False)

    
    if(len(X_cat_)>0):
        
        bar_color = get_bar_colors(data, X_cat_)
        
        plt.title(title)
        sns.barplot(x="Estimate", y="Features", data = data, xerr=data['Estimate std'], palette =bar_color)
        plt.xlim(xlim)
    else:
        plt.title(title)
        sns.barplot(x="Estimate", y="Features", data = data, color = 'darkblue')
        plt.xlim(xlim)


    y_step=0  
    for i in range(df.shape[0]):
        if(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.01):
            color = 'green'
        elif(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.05):
            color = 'gray'
        else:
            color = 'red'        
        
        if(pvalue_type == 'color'):
                plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]-0.005, y_step, 
                                 '' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],2)),
                                 size= size, rotation=0.,
                                 ha="left", va="center", color = 'white',
                                 bbox=dict(boxstyle="round",
                                           ec=color,
                                            fc=color,
                                           )
                                 )
                y_step+=1

        elif(pvalue_type == 'value'):
                plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                 '(' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],8))+')',
                                 size= size, rotation=0.,
                                 ha="left", va="center", color = 'black',

                                 )
                y_step+=1
        else:
            
            if(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]<0.01):
                
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '**',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )
            elif(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]>= 0.01 and df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]<0.05):
                  
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '*',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )  
            else:
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     ' ',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )    

            y_step+=1
            
         
    text_diff =xlim[1]
    #coef_SVR_tot.loc['R-squared', 'value']
    f_prob = '(ns)'
    if(regression_model.loc['F-pvalue', 'value']<0.01):
        f_prob = '(**)'
    elif(regression_model.loc['F-pvalue', 'value']<0.05):
        f_prob = '(*)'
        
    
    plot_ci = np.round(regression_model.loc['CI', 'value'],2)
    
    plt.text(report_pos[0], report_pos[1],
             r'$ R^2 = $' + str(np.round(regression_model.loc['R-squared', 'value'],2))+ '\n$CI = $' + str(plot_ci) + '\n$F^2 = $' + str(np.round(regression_model.loc['F-squared', 'value'],2)) + ' ' + '\n$F = $' + str(np.round(regression_model.loc['F', 'value'],2)) + '\n$P (F) = $' + f_prob + '\n$n = $' + n,
                             size= 12, rotation=0.,
                             ha="right", va="center", color = 'black',
                             bbox=dict(boxstyle="round",
                                       ec='#F6F6F6',
                                        fc='#F6F6F6',
                                       )
                             )

    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    
    plt.xlabel('Estimate', fontsize=12)
    plt.ylabel('Features', fontsize=12)
    plt.locator_params(axis='x', nbins=4)


def adj_r2_score_and_r2_score(clf, X, y):
    n = X.shape[0]  # Number of observations
    p = X.shape[1]  # Number of features
    r_squared = r2_score(y, clf.predict(X))
    return [1 - (1 - r_squared) * ((n - 1) / (n - p - 1)), r_squared]


def mse(clf, X, y):
    return mean_squared_error(y, clf.predict(X))

def rmse(clf, X, y):
    mse = mean_squared_error(y, clf.predict(X))
    return math.sqrt(mse)    

def coef_se(clf, X, y):
    n = X.shape[0]
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y, clf.predict(X)) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)

def coef_tval(clf, X, y):
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf.coef_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)

def coef_tval_SVR_tree(clf, X, y):
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf._get_coef() / coef_se(clf, X, y)[1:])
    return np.append(a, b)

def coef_pval(clf, X, y):

    n = X.shape[0]
    t = coef_tval(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def coef_pval_SVR_tree(clf, X, y):

    n = X.shape[0]
    t = coef_tval_SVR_tree(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def residuals(clf, X, y, r_type='standardized'):

    # Make sure value of parameter 'r_type' is one we recognize
    assert r_type in ('raw', 'standardized', 'studentized'), (
        "Invalid option for 'r_type': {0}".format(r_type))
    y_true = y.view(dtype='float')
    # Use classifier to make predictions
    y_pred = clf.predict(X)
    # Make sure dimensions agree (Numpy still allows subtraction if they don't)
    assert y_true.shape == y_pred.shape, (
        "Dimensions of y_true {0} do not match y_pred {1}".format(y_true.shape,
                                                                  y_pred.shape))
    # Get raw residuals, or standardized or standardized residuals
    resids = y_pred - y_true
    if r_type == 'standardized':
        resids = resids / np.std(resids)
    elif r_type == 'studentized':
        # Prepare a blank array to hold studentized residuals
        studentized_resids = np.zeros(y_true.shape[0], dtype='float')
        # Calcluate hat matrix of X values so you can get leverage scores
        hat_matrix = np.dot(
            np.dot(X, np.linalg.inv(np.dot(np.transpose(X), X))),
            np.transpose(X))
        # For each point, calculate studentized residuals w/ leave-one-out MSE
        for i in range(y_true.shape[0]):
            # Make a mask so you can calculate leave-one-out MSE
            mask = np.ones(y_true.shape[0], dtype='bool')
            mask[i] = 0
            loo_mse = np.average(resids[mask] ** 2, axis=0)  # Leave-one-out MSE
            # Calculate studentized residuals
            studentized_resids[i] = resids[i] / np.sqrt(
                loo_mse * (1 - hat_matrix[i, i]))
        resids = studentized_resids
    return resids


def f_squared(clf, X, y):

    n = X.shape[0]
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    return r_squared  / (1 - r_squared)

def f_stat(clf, X, y):
    """Calculate summary F-statistic for beta coefficients.

    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].

    Returns
    -------
    float
        The F-statistic value.
    """
    n = X.shape[0]
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    return (r_squared / p) / ((1 - r_squared) / (n - p - 1))


def f_stat_pvalue(clf, X, y):
    """Calculate summary F-statistic p value for beta coefficients.

    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].

    Returns
    -------
    float
        The F-statistic p value.
    """
    n = X.shape[0] # Esto se extrae par los grados de libertad del numeador y el denomindor (no. predictores, no. sujetos - no. predictores-1)
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    
    return np.round(scipy.stats.f.sf(f_stat(clf, X, y), n, (n - p - 1)), 15)

def compute_f_statistics(clf, X, y):
    return [f_stat(clf, X, y), f_stat_pvalue(clf, X, y)]



def boostrapping(X_func, case1A_func, case1A_func_for_strat, lista_best_func, model_ = SVR(kernel='rbf', degree=4)):
    print(model_)
    
    coef_array_func = np.zeros([len(lista_best_func)+1, 1000])
    coef_t_value_func = np.zeros([len(lista_best_func)+1, 1000])
    coef_p_value_func = np.zeros([len(lista_best_func)+1, 1000])

    r2_list_func =[]
    f2_list_func =[]
    f_list_func =[]
    f_p_value_list_func =[]
    for iter in tqdm(range(0,1000), total=len(range(0, 1000)),leave=True, mininterval=1, ascii=True,
                            colour='green', desc='Training model'):
        # Stratified split
        seed = np.random.randint(100, 200)

        X_train, X_test, y_train, y_test = train_test_split(X_func, case1A_func, stratify=case1A_func_for_strat,test_size=0.30, random_state=seed)

        model_func = model_

        model_func.fit(X_train, y_train)

        coef_array_func[0,iter] = np.abs(model_func.intercept_)
        coef_array_func[1::,iter] = np.abs(model_func._get_coef())


        coef_t_value_func[:,iter] = np.round(coef_tval_SVR_tree(model_func, X_test, y_test), 30)
        coef_p_value_func[:,iter] = np.round(coef_pval_SVR_tree(model_func, X_test, y_test), 30)


        f2_list_func.append(f_squared(model_func, X_test, y_test))
        f_list_func.append(f_stat(model_func, X_test, y_test))
        f_p_value_list_func.append(f_stat_pvalue(model_func, X_test, y_test))
        r2_list_func.append(r2_score(y_test, model_func.predict(X_test)))


    r2_list_array = np.array(r2_list_func)
    # finding the 1st quartile
    q1 = np.quantile(r2_list_array, 0.25)
 
    # finding the 3rd quartile
    q3 = np.quantile(r2_list_array, 0.75)
    med = np.median(r2_list_array)

    # finding the iqr region
    iqr = q3-q1

    # finding upper and lower whiskers
    upper_bound = q3+(1.5*iqr)
    lower_bound = q1-(1.5*iqr)
    #print(iqr, upper_bound, lower_bound)

    index_del = []     

    for i in range(r2_list_array.shape[0]):
        if((r2_list_array[i]<= lower_bound) | (r2_list_array[i] >= upper_bound)):
            index_del.append(i)

    
    
    r2_list_func = list(np.delete(r2_list_array,index_del, axis=0 ))

    coef_df_func = pd.DataFrame(
            index=['_intercept'] + lista_best_func,
            columns=['Estimate','t value', 'p value', 'p value mean']
        )

    coef_array_func  = np.delete(coef_array_func,index_del, axis=1 )
    coef_t_value_func = np.delete(coef_t_value_func,index_del, axis=1 )
    coef_p_value_func = np.delete(coef_p_value_func,index_del, axis=1 )
    
    
    coef_array_func_mean = np.zeros([len(lista_best_func)+1, 1])
    coef_t_value_func_mean = np.zeros([len(lista_best_func)+1, 1])
    coef_p_value_func_mean = np.zeros([len(lista_best_func)+1, 1])
    coef_p_value_func_fdr = np.zeros([len(lista_best_func)+1, 1])
    
    coef_array_func_std = np.zeros([len(lista_best_func)+1, 1])

    print('shape',coef_array_func.shape, coef_t_value_func.shape, coef_p_value_func.shape, len(index_del))
    
    for i in range(len(lista_best_func)+1):
        coef_array_func_mean[i] = coef_array_func[i,:].mean()
        coef_t_value_func_mean[i]= coef_t_value_func[i,:].mean()
        coef_p_value_func_mean[i]= coef_p_value_func[i,:].mean()
        coef_p_value_func_fdr[i]= fdrcorrection(coef_p_value_func[i,:], alpha=0.05, is_sorted=False )[1].max()
        
        coef_array_func_std[i] = coef_array_func[i,:].std()
        # Se corrige por FDR y se reporta el valor máximo

    coef_df_func['Estimate'] = coef_array_func_mean
    coef_df_func['t value'] = coef_t_value_func_mean
    coef_df_func['p value'] = coef_p_value_func_fdr
    coef_df_func['p value mean'] = coef_p_value_func_mean
    coef_df_func['Estimate std'] = coef_array_func_std


    empty_str = []
    for i in range(coef_df_func.shape[0]):
        empty_str.append('')

    coef_df_func['value'] = empty_str

    coef_df_func = coef_df_func.T
    coef_df_func['R-squared'] = ['','','','', '', np.mean(r2_list_func)]
    coef_df_func['CI'] = ['','','','', '', np.std(r2_list_func)*1.96]
    coef_df_func['F-squared'] = ['','','','', '', np.mean(f2_list_func)]
    coef_df_func['F'] = ['','','','', '', np.mean(f_list_func)]
    coef_df_func['F-pvalue'] = ['','','','', '', fdrcorrection(f_p_value_list_func, alpha=0.05, is_sorted=False )[1].max()]
    
    #coef_df_func['F-pvalue_fdr'] = ['','','', fdrcorrection(f_p_value_list_func, alpha=0.05, method='indep', is_sorted=False )]
    coef_df_func = coef_df_func.T


    print(np.mean(r2_list_func),  np.std(r2_list_func)*1.96)
    
    return [coef_df_func, r2_list_func, f_p_value_list_func, fdrcorrection(f_p_value_list_func, alpha=0.05, is_sorted=False )]

   
    
def SearchBestModel(case_x, case_y, case_x_for_strat):
    X_train, X_test, y_train, y_test = train_test_split(case_x, case_y, test_size=.30, stratify=case_x_for_strat, random_state=1)

    opt_SVR = BayesSearchCV(
            SVR(),
            {
                #'gamma': ( 0.0001, 0.01, 0.001, 0.1, 1, 10),
                'kernel': ['rbf'],
                'degree': (3, 4, 5, 6),
                #'tol:' : (1e-6, 1e-3, 1e+1),
                #'n_estimators': (100, 1000),

            },
            n_iter=10,
            cv=3, random_state=1
        )

    opt_SVR.fit(X_train, y_train)


    print('SVR')
    print("val. score: %s" % opt_SVR.best_score_)
    print("test score: %s" % opt_SVR.score(X_test, y_test))
    print("best parameters: %s" % str(opt_SVR.best_params_))
    print('---------------------------------------------\n')
    
    return opt_SVR

## Colors, variables names

In [3]:
X_cat = pd.read_csv('../Data/var_name_color_code.csv', encoding='latin-1', sep=";")

## Load Data and z-score

In [4]:
data_ = pd.read_excel('../Data/Base.xlsx', index_col=0)

In [5]:
colunm = 'diagnosis_specify'
cn_col = data_[colunm]

data_[colunm] = data_[colunm].apply(lambda x: (x-cn_col.mean())/ cn_col.std())


colunm = 'country_class'
cn_col = data_[colunm]

data_[colunm] = data_[colunm].apply(lambda x: (x-cn_col.mean())/ cn_col.std())

colunm = 'Edad_procesada'
cn_col = data_[colunm]

data_[colunm] = data_[colunm].apply(lambda x: (x-cn_col.mean())/ cn_col.std())


colunm = 'years_education'
cn_col = data_[colunm]

data_[colunm] = data_[colunm].apply(lambda x: (x-cn_col.mean())/ cn_col.std())

colunm = 'cognition_general_score'
cn_col = data_[colunm]

data_[colunm] = data_[colunm].apply(lambda x: (x-cn_col.mean())/ cn_col.std())


colunm = 'executive_score'
cn_col = data_[colunm]

data_[colunm] = data_[colunm].apply(lambda x: (x-cn_col.mean())/ cn_col.std())

In [6]:
conn_z_score = data_.copy()

In [7]:
conn_z_score.reset_index(inplace=True, drop=True)

## Results

### Demographics and cognition

In [8]:
data_reduc_1A_zscore = conn_z_score.copy()

In [9]:
conn_z_score_fer = data_reduc_1A_zscore.drop([              'mini_sea_tom', 'mini_sea_total'], axis =1 )
conn_z_score_tom = data_reduc_1A_zscore.drop(['mini_sea_fer',                 'mini_sea_total'], axis =1 )
conn_z_score_tot = data_reduc_1A_zscore.drop(['mini_sea_fer', 'mini_sea_tom',                 ], axis =1 )


In [10]:
conn_z_score_fer.dropna(inplace=True)
conn_z_score_tom.dropna(inplace=True)  
conn_z_score_tot.dropna(inplace=True) 


In [11]:
print(conn_z_score_fer.shape,
conn_z_score_tom.shape,  
conn_z_score_tot.shape)

(998, 8) (998, 8) (998, 8)


In [12]:
## y data
y_1A_fer = conn_z_score_fer['mini_sea_fer']
y_1A_tom = conn_z_score_tom['mini_sea_tom']
y_1A_tot = conn_z_score_tot['mini_sea_total']


## x data
X_1A_fer = conn_z_score_fer.drop(['mini_sea_fer'], axis =1 )
X_1A_tom = conn_z_score_tom.drop(['mini_sea_tom'], axis =1 )
X_1A_tot = conn_z_score_tot.drop(['mini_sea_total'], axis =1 )



### Boostrapping and Regression Models

In [13]:
y_1A_fer_for_strat =  y_1A_fer.copy()
median = y_1A_fer_for_strat.median()
y_1A_fer_for_strat.loc[y_1A_fer_for_strat<median]= 0
y_1A_fer_for_strat.loc[y_1A_fer_for_strat>=median]=1 


opt_SVR_fer = SearchBestModel(X_1A_fer, y_1A_fer, y_1A_fer_for_strat)


SVR
val. score: 0.3342902558222461
test score: 0.32021944179817374
best parameters: OrderedDict([('degree', 5), ('kernel', 'rbf')])
---------------------------------------------



In [14]:
y_1A_tom_for_strat =  y_1A_tom.copy()
median = y_1A_tom_for_strat.median()
y_1A_tom_for_strat.loc[y_1A_tom_for_strat<median]= 0
y_1A_tom_for_strat.loc[y_1A_tom_for_strat>=median]=1 


opt_SVR_tom = SearchBestModel(X_1A_tom, y_1A_tom, y_1A_tom_for_strat)

SVR
val. score: 0.3520572199003
test score: 0.26295314528654523
best parameters: OrderedDict([('degree', 5), ('kernel', 'rbf')])
---------------------------------------------



In [15]:
y_1A_tot_for_strat =  y_1A_tot.copy()
median = y_1A_tot_for_strat.median()
y_1A_tot_for_strat.loc[y_1A_tot_for_strat<median]= 0
y_1A_tot_for_strat.loc[y_1A_tot_for_strat>=median]=1 

opt_SVR_tot = SearchBestModel(X_1A_tot, y_1A_tot, y_1A_tot_for_strat)

SVR
val. score: 0.39521707611199836
test score: 0.4733987206514443
best parameters: OrderedDict([('degree', 5), ('kernel', 'rbf')])
---------------------------------------------



## Boostrapping

In [16]:
coef_SVR_fer = boostrapping(X_1A_fer, y_1A_fer, y_1A_fer_for_strat, list(X_1A_fer.columns))
coef_SVR_tom = boostrapping(X_1A_tom, y_1A_tom, y_1A_tom_for_strat, list(X_1A_tom.columns))
coef_SVR_tot = boostrapping(X_1A_tot, y_1A_tot, y_1A_tot_for_strat, list(X_1A_tot.columns))


SVR(degree=4)


Training model:   0%|          | 0/1000 [00:00<?, ?it/s]

shape (8, 1000) (8, 1000) (8, 1000) 0
0.34586585977296963 0.07434941211475332
SVR(degree=4)


Training model:   0%|          | 0/1000 [00:00<?, ?it/s]

shape (8, 1000) (8, 1000) (8, 1000) 0
0.3399315677373424 0.08521541918695502
SVR(degree=4)


Training model:   0%|          | 0/1000 [00:00<?, ?it/s]

shape (8, 982) (8, 982) (8, 982) 18
0.4419682105811034 0.0656704517800694


### Save results

In [18]:
#coef_SVR_fer[0].to_excel('Results/Fig_2_panel_1/fer.xlsx')
#coef_SVR_tom[0].to_excel('Results/Fig_2_panel_1/tom.xlsx')
#coef_SVR_tot[0].to_excel('Results/Fig_2_panel_1/tot.xlsx')